In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.6.0


## 예제 데이터 셋 받기
- MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. 
- 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1,000개만 사용겠습니다:

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 모델 정의
- 간단한 모델 만들기

In [3]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기
- 훈련된 모델을 다시 훈련할 필요 없이 사용하거나 훈련 과정이 중단될 경우 중단 이후 부분에서 다시 훈련 시작할 수있음
- tf.keras.callbacks.ModelCheckpoint 콜백을 사용하면 훈련 도중 또는 훈련 종료 시 모델을 지속적으로 저장할 수 있음

## 체크포인트 콜백 사용하기
- 훈련하는 동안 가중치를 저장하기 위해 ModelCheckpoint 콜백 만들기

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 2s 9ms/step - loss: 1.1708 - accuracy: 0.6780 - val_loss: 0.6982 - val_accuracy: 0.7980

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 5ms/step - loss: 0.4163 - accuracy: 0.8870 - val_loss: 0.5197 - val_accuracy: 0.8400

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2766 - accuracy: 0.9310 - val_loss: 0.4662 - val_accuracy: 0.8510

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2164 - accuracy: 0.9490 - val_loss: 0.4396 - val_accuracy: 0.8640

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 5ms/step - loss: 0.1472 - accuracy: 0.9710 - val_loss: 0.4479 - val_accuracy: 0.8540

Epoch 00005: saving model to training_1\cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s 

### 이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트를 함

In [5]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

### 두 모델이 동일한 아키텍처를 공유하기만 한다면 두 모델 간에 가중치를 공유할 수 있습니다. 
### 따라서 가중치 전용에서 모델을 복원할 때 원래 모델과 동일한 아키텍처로 모델을 만든 다음 가중치를 설정합니다.

### 이제 훈련되지 않은 새로운 모델을 다시 빌드하고 테스트 세트에서 평가합니다. 훈련되지 않은 모델은 확률 수준(~10% 정확도)에서 수행됩니다.

In [6]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3293 - accuracy: 0.1300
훈련되지 않은 모델의 정확도: 13.00%


In [8]:
# 체크포인트에서 가중치를 로드하고 다시 평가

# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4198 - accuracy: 0.8590
복원된 모델의 정확도: 85.90%


## 체크포인트 콜백 매개변수
- 콜백함수는 몇가지 매개변수를 제공함
1. 체크포인트 이름을 고유하게 만들기
2. 체크포인트 주기 조정하기

In [9]:
# 체크포인트 고유 이름으로 저장하기

# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5) # 저장 주기 설정

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [10]:
# 만들어진 체크포인트 확인 및 마지막 체크포인트 선택
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

### 참고: 기본 TensorFlow 형식은 가장 최근의 체크포인트 5개만 저장합니다.

### 모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보겠습니다:

In [12]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4934 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


## 위 파일들에 대해
### 위의 코드는 이진 형식의 훈련된 가중치만 포함하는 체크포인트 형식의 파일 모음에 가중치를 저장함
### 체크포인트 포함항목
1. 모델의 가중치를 포함하는 하나 이상의 샤드
2. 어떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일

### 단일 머신에서 모델을 훈련할 경우 접미사가 .data-00000-of-00001 인 하나의 샤드를 갖게 됨

## 수동으로 가중치 저장하는 방법
### Model.save_weights 메서드를 사용하여 수동으로 가중치를 저장합니다. 
### 기본적으로 tf.keras, 특히 save_weights는 .ckpt 확장자가 있는 TensorFlow 체크포인트 형식을 사용합니다
### (.h5 확장자를 사용하여 HDF5에 저장하는 내용은 모델 저장 및 직렬화 가이드에서 다룸).

In [13]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4934 - accuracy: 0.8730
복원된 모델의 정확도: 87.30%


## 전체 모델 저장하기
- model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다.
- 모델을 저장하기 때문에 원본 파이썬 코드*가 없어도 사용할 수 있습니다. 
- 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.

- 전체 모델은 두 가지 다른 파일 형식(SavedModel 및 HDF5)으로 저장할 수 있습니다.
- TensorFlow SavedModel 형식은 TF2.x의 기본 파일 형식입니다.
- 그러나 모델을 HDF5 형식으로 저장할 수 있습니다.
- 전체 모델을 두 가지 파일 형식으로 저장하는 방법에 대한 자세한 내용은 아래에 설명되어 있습니다.

- 전체 모델을 저장하는 기능은 매우 유용합니다.
- TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다(Saved Model, HDF5).
- 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다(Saved Model, HDF5).

- 사용자 정의 객체(예를 들면 상속으로 만든 클래스나 층)는 저장하고 로드하는데 특별한 주의가 필요합니다.
- 아래 사용자 정의 객체 저장하기 섹션을 참고하세요.

## SavedModel 포맷
- SavedModel 형식은 모델을 직렬화하는 또 다른 방법입니다.
- 이 형식으로 저장된 모델은 tf.keras.models.load_model을 사용하여 복원할 수 있으며 TensorFlow Serving과 호환됩니다.
- SavedModel 가이드에 SavedModel을 제공/검사하는 방법이 자세히 설명되어 있습니다.
- 아래 섹션은 모델을 저장하고 복원하는 단계를 보여줍니다.

In [14]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1691 - accuracy: 0.6790
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4310 - accuracy: 0.8900
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2910 - accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2176 - accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1598 - accuracy: 0.9700
INFO:tensorflow:Assets written to: saved_model/my_model\assets


SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함하는 디렉토리입니다. 저장된 모델 디렉토리를 검사합니다.

In [18]:
# my_model 디렉토리
%ls saved_model

 D ����̺��� ����: HDD
 ���� �Ϸ� ��ȣ: 604B-CE05

 d:\Workspace\03.UseDataAnalysis\03.�˰����� ��� ���н�\saved_model ���͸�

2021-10-11  ���� 11:55    <DIR>          .
2021-10-11  ���� 11:55    <DIR>          ..
2021-10-11  ���� 11:55    <DIR>          my_model
               0�� ����                   0 ����Ʈ
               3�� ���͸�  955,339,350,016 ����Ʈ ����


In [19]:
# assests 폴더, saved_model.pb, variables 폴더
%ls saved_model/my_model

����ġ�� Ʋ���ϴ� - "my_model".


In [20]:
# 저장된 모델로 부터 새로운 케라스 모델 로드
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [21]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4427 - accuracy: 0.8530
복원된 모델의 정확도: 85.30%
(1000, 10)


### HDF5 파일로 저장하기
- 케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [22]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.2137 - accuracy: 0.6510
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4119 - accuracy: 0.9010
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2987 - accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2159 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1618 - accuracy: 0.9640


h5 파일로 모델 다시 만들기

In [23]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
# 정확도 확인
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4052 - accuracy: 0.8540
복원된 모델의 정확도: 85.40%


## Keras는 아키텍처를 검사하여 모델을 저장합니다. 이 기술은 모든 내용을 저장합니다.

- 가중치 값 (w + b)
- 모델 구조
- 모델의 훈련 구성(.compile() 메서드에 전달하는 내용)
- 존재하는 옵티마이저와 그 상태(훈련을 중단한 곳에서 다시 시작할 수 있게 해줌)

### 체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(tf.compat.v1.train)를 저장할 수 없습니다. 
### v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다.
### 따라서 옵티마이저의 상태를 잃게 됩니다.

## 사용자 정의 객체
### SavedModel 형식을 사용하는 경우, 이 섹션을 건너뛸 수 있습니다.
### HDF5와 SavedModel의 주요 차이점은 HDF5는 객체 구성을 사용하여 모델 아키텍처를 저장하는 반면,
### SavedModel은 실행 그래프를 저장한다는 것입니다.
### 따라서 SavedModel은 원본 코드 없이도 서브클래싱된 모델 및 사용자 지정 레이어와 같은 사용자 지정 객체를 저장할 수 있습니다.

> 사용자 정의 객체를 HDF5로 저장하려면 다음 과정을 따르세요:

- 이 객체에 get_config 메서드를 정의하고 선택적으로 from_config 클래스 메서드를 정의합니다.
- get_config(self)는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
- from_config(cls, config)는 get_config에서 반환된 설정을 사용해 새로운 객체를 만듭니다.
- 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(return cls(**config)).
- 모델을 로드할 때 이 객체를 custom_objects 매개변수로 전달합니다.
- 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕서너리를 매개변수로 제공해야 합니다.
- 예를 들면 tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})
- 사용자 정의 객체와 get_config에 관한 예제를 보려면 Writing layers and models from scratch 튜토리얼을 참고하세요.